## Duygu Analizi Nedir?
Duygu Analizi ile, örneğin bir konuşmacının veya yazarın bir belgeye, etkileşime ya da olaya ilişkin tutumunu (örneğin duyarlılık) belirlemek istiyoruz. Bu nedenle, metnin anlaşılması gereken, altında yatan amacı öngörmek doğal bir dil işleme sorunudur. Duygu çoğunlukla olumlu, olumsuz ve tarafsız kategorilere ayrılır. Duygu Analizi'nin kullanılmasıyla, örneğin bir müşteri hakkında, onun hakkında yazdığı incelemeye dayanarak, bir ürün hakkındaki görüş ve tutumlarını tahmin etmek istiyoruz. Bu nedenle, Duygu Analizi, incelemeler, anketler, belgeler ve daha pek çok şeye yaygın olarak uygulanır.

## İmdb Veri Kümesi
İmdb duyarlılık sınıflandırma veri seti, pozitif (1) veya negatif (0) olarak etiketlenmiş imdb kullanıcılarının 50.000 film incelemesinden oluşur. İncelemeler önceden işlenir ve her biri tam sayı biçiminde bir sözcük dizini dizisi olarak kodlanır. İncelemelerdeki kelimeler, veri kümesi içindeki genel sıklıklarına göre dizine eklenir. Örneğin, “2” tamsayısı, verilerdeki ikinci en sık kullanılan sözcüğü kodlar. 50.000 inceleme, eğitim için 25.000 ve test için 25.000’e bölünmüştür. Veri seti, Stanford Üniversitesi araştırmacıları tarafından oluşturulmuştur ve 2011'de % 88.89 doğruluk elde ettikleri bir bildiri ile yayınlanmıştır. 2011'de “Bag of Words Meets Bags of Popcorn” adlı Kaggle yarışmasında da kullanılmıştır.

### Gerekli Temel Kütüphaneler

In [3]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from keras import models
from keras import layers
!pip install numpy==1.16.1
import numpy as np

### Önce IMDB Veri setimizi indirelim daha sonra Train ve Test verilerimizi birleştirelim

In [0]:
import numpy as np
from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [6]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))

length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Categories: [0 1]
Number of unique words: 9998
Average Review length: 234.75892
Standard Deviation: 173.0


Yukarıdaki çıktıda veri kümesinin, incelemenin duyarlılığını temsil eden 0 veya 1 olmak üzere iki kategoride etiketlendiğini görebilirsiniz. Veri kümesinin tamamı 9998 benzersiz kelime içeriyor ve ortalama inceleme uzunluğu 233 kelime, standart 173 kelime sapma ile.

In [7]:
print("Label:", targets[0])

Label: 1


In [8]:
print(data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Yukarıda, pozitif olarak etiketlenen veri kümesinin ilk incelemesini görüyorsunuz (1). Aşağıdaki kod, sözlük eşleme kelime endekslerini orijinal kelimelere geri getirir, böylece bunları okuyabiliriz. Bilinmeyen her kelimeyi "#" ile değiştirir. Bunu get_word_index () işlevini kullanarak yapar .

In [9]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded) 

1646592/1641221 [==============================] - 1s 0us/step
# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amaz

## Veri Hazırlama
Şimdi verilerimizi hazırlama zamanı. Her incelemeyi vektörleştireceğiz ve tam olarak 10.000 sayı içerecek şekilde sıfırlarla dolduracağız. Bu, 10.000'den kısa her incelemeyi sıfırlarla doldurduğumuz anlamına gelir. Bunu yapıyoruz çünkü en büyük inceleme neredeyse o kadar uzun ve sinir ağımız için her girdi aynı boyutta olmalı. Ayrıca hedefleri yüzer hale getiriyoruz.

In [0]:
def vectorize(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results
 
data = vectorize(data)
targets = np.array(targets).astype("float32")

Şimdi verilerimizi bir eğitim ve test setine ayırdık. Eğitim seti 40.000 gözden geçirmeyi ve test seti 10.000'i içerecek.

In [0]:
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

## Model Oluşturma ve Eğitim

In [19]:
from keras.models import Sequential

# Input - Layer
model = Sequential()
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                500050    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)      

binary_crossentropy ve Adam optimizer ile compile edelim

In [0]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [21]:
results = model.fit(
 train_x, train_y,
 epochs= 2,
 batch_size = 500,
 validation_data = (test_x, test_y)
)

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 6s 147us/step - loss: 0.4044 - acc: 0.8218 - val_loss: 0.2636 - val_acc: 0.8949
Epoch 2/2
40000/40000 [==============================] - 3s 74us/step - loss: 0.2122 - acc: 0.9184 - val_loss: 0.2598 - val_acc: 0.8948


Modelimizi değerlendirme zamanı:

In [22]:
print(np.mean(results.history["val_acc"]))

0.8948500022292136


Bu basit modelle, başlangıçta bahsettiğim 2011 makalesinin doğruluğunu çoktan aştık. Hiperparametreleri ve katman sayısını denemekten çekinmeyin.
### Faydalı olması dileğiyle